In [64]:
import os
import subprocess
import re
import pandas as pd
import numpy as np
#import nltk
import spacy
from spacy.matcher import Matcher
#from nltk.corpus import stopwords
nlp = spacy.load("en_core_web_sm")
matcher=Matcher(nlp.vocab)
import PyPDF2
from pywintypes import com_error
import win32com.client as win32
import io
import docx2txt
from bs4 import BeautifulSoup
import unicodedata

In [65]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import spacy

In [66]:
path = "C:\\Users\\Rohit Peter\\res\\"
for files in os.listdir(path):
    for filename in files:
        if filename.endswith(('.pdf')):
            abspath = os.path.join(path, filename)
            subprocess.call('lowriter --invisible --convert-to doc "{}"'
                            .format(abspath), shell=True)

In [67]:
def extract_names(document):
    nlp_text=nlp(document)
    pattern=[{'POS':'PROPN'},{'POS':'PROPN'}]
    pattern1=[{'POS':'PROPN'},{'POS':'PROPN'},{'POS':'PROPN'}]
    matcher.add('NAME',None,pattern)
    matches=matcher(nlp_text)
    for match_id,start,end in matches:
        span=nlp_text[start:end]
        if len(span.text) < 3:
            pattern1=[{'POS':'PROPN'},{'POS':'PROPN'},{'POS':'PROPN'}]
            matcher.add('NAME',None,pattern1)
            matches1=matcher(nlp_text)
            for match_id,start,end in matches1:
                span=nlp_text[start:end]                
        return span.text

In [68]:
def exper(fullText):
    mi=fullText.lower()
    #print(mi)
    h=mi.replace("_"," ")
    h=h.replace("-"," ")
    h=h.replace(","," ")
    h=h.replace("("," ")
    h=h.replace(")"," ")
    h=h.replace(".docx"," ")
    h=h.replace(".pdf"," ")
    h=h.split()              #look at h only years get it
    if 'years' in h and 'months' in h:
        if h[h.index('years')-1].isdigit() ==True:
            d=h[h.index('years')-1] + " " + h[h.index('years')]+ " " +h[h.index('months')-1] + " " +h[h.index('months')]
        else:
            if int(h[h.index('months')-1]) >= 12:
                d = str(int(h[h.index('months')-1])/12) + " " + h[h.index('years')]
                print('yay!')
    #elif 'year' in h:
        #d=h[h.index('year')-1] + " " + h[h.index('year')]
    elif 'years' in h:
        d=h[h.index('years')-1] + " " + h[h.index('years')]
    elif 'months' in h:
        if int(h[h.index('months')-1]) >= 12:
            d = str(int(h[h.index('months')-1])/12) + " " + h[h.index('years')]
            #print('yay!')
        else:
            d =h[h.index('months')-1] + " " + h[h.index('months')]
    #elif 'month' in h:
     #   d=h[h.index('month')-1] + " " + h[h.index('month')]
    #elif re.search('no experience',str(h),re.M|re.I) :
        #d='No Experience'
    else:
        #d=generate_ngrams(fullText, 2)  
        d='No Experience'
    return d   

In [69]:
def extract_text_from_pdf(path, filename):
    with open(path+filename, 'rb') as fh:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            #clean_text = unicodedata.normalize("NFKD",text)
            line = ' '.join(text.split())
            #yield clean_text
            #yield text
            yield line

            # close open handles
            converter.close()
            fake_file_handle.close()



In [70]:
k = []

In [71]:
k.append({"Name":'asdf',"exp":"123"})

In [72]:
k

[{'Name': 'asdf', 'exp': '123'}]

In [73]:
path = 'C:\\Users\\Rohit Peter\\res\\'
#data = []
data = pd.DataFrame(columns = ["Name","Experience"])
n = []
e = []
text11 = []
####IMPORTING PDF ONLY
for filename in os.listdir(path):
    if filename.endswith(('.pdf')):
        print(filename)
        res = []
        for page in extract_text_from_pdf(path, filename):
            
            res +="" + page
            res = ''.join(res)
            res = res.replace('\n', ' ')
            res = res.strip()
            text11+= [res]
            
        name = extract_names(res)
        print(name,len(name))
        n+=[name]
        exp = exper(res)
        print(exp)
        e+=[exp]
        #data.append({"Name":name,"Experience": exp},ignore_index=True)
        data._update_inplace(data.append([[name,exp]]))
        df = pd.DataFrame(list(zip(n,e)), columns = ["Name","Experience"])    
        df.to_csv("resumes_filter10.csv", index=False)

Ashis_3yr_Infosys.pdf
ASHIS PANDA 11
3+ years
Ayan_cv_pdf.pdf
AYAN MAZUMDER 13
yay!
3.0 years


C:\Users\Rohit Peter\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


DilipBehera_Resume.pdf
Dilip Kumar 11
3.5 years
Jagruti J Resume.pdf
Jagruti Khatri 14
No Experience
Kuntal RoyChowdhury_Resume.pdf
KUNTAL ROY 10
No Experience
Megha_Resume.pdf
Megha Kumari 12
No Experience
Resume (1).pdf
Akshay K 8
No Experience


In [74]:
df

,Name,Experience
0,ASHIS PANDA,3+ years
1,AYAN MAZUMDER,3.0 years
2,Dilip Kumar,3.5 years
3,Jagruti Khatri,No Experience
4,KUNTAL ROY,No Experience
5,Megha Kumari,No Experience
6,Akshay K,No Experience


In [169]:
text11

['5/27/2019 ASHIS PANDA DATA ANALYST ashisk.akp@gmail.com +917358270790 ashiskumarpanda \uf0e0 \uf0ac https://medium.com/@GeneAshis \uf095 \uf099 \uf0e1 https://www.linkedin.com/in/ashis- panda-b211b592/ CaptainAshis \uf09b Summary Data Scientist with 3+ years of Experience executing data-driven solution to increase e\x00ciency and accuracy. Experienced in creating machine learning models using predictive data modelling techniques and analyzing the output of the algorithm to deliver insights and implement action oriented solutions to complex business problems. Skills PYTHON SQL R DATA SCIENCE MACHINE LEARNING ALGORITHMS TABLEAU PYTORCH KERAS MATPLOTLIB NUMPY PANDAS DEEP LEARNING SCIKIT LEARN https://resume.creddle.io Education Creddle | Resume Bachelor Of Technology in Electrical and Electronics Engineering at Vellore Institute of Technology , India 2011 to 2015 Employment Infosys Analyst Projects Bangalore Dec. 2015 to Current Developed the Dynamic pricing module for one of the larges

In [13]:
tt1 = 'Dear Parent,\xa0This is a test message,\xa0kindly ignore it.\xa0Thanks'

In [14]:
clean_text = BeautifulSoup(tt1, "lxml").get_text(strip=True)

In [17]:
clean_text1 =  unicodedata.normalize("NFKD",tt1)

In [18]:
clean_text1

'Dear Parent, This is a test message, kindly ignore it. Thanks'

In [21]:
clean_text

'Dear Parent,\xa0This is a test message,\xa0kindly ignore it.\xa0Thanks'

In [55]:
text21 = "Asha_Resume.pdf [' Asha H R \n\nMobile: +91 9538044096 \n\nE-Mail: ashahr.jc@gmail.com \n\nStatistical Analysis, Reporting, Market Research and Digital Marketing \n\n \nCareer Objective: \n \nTo secure a responsible position in Statistical Analytics/Modelling/SQL reporting where, I may utilize my professional skills and \npersonal capabilities towards enhancing the growth of the organization. \n \nProfile Summary: \n•  M.Sc. in Statistics with 3.2 years of experience in Statistical Analysis, Market Research and SQL reporting with Nielsen and \n\nBASES and 1 year of experience in online media reporting as a Campaign specialist with Gamut. \n\n•  Have proficient knowledge in analysing the data using some statistical tools like MS Excel, MS Word, MS Access PL/SQL and \n\nBase SAS and digital marketing tools like DFA, DFP, Flashtalking, Sizmek, Xad and TTD. \n \n\nTechnical Skillset: \n \n•  Working knowledge on Statistics concept, SAS, PL/SQL, and SPSS  \n•  Well versed with MS Office Suite (Word, Excel, outlook, Access& PowerPoint), Computer Hardware,  \n• \n•  Worked on digital marketing portals such as, DFA - Tool, Google docs, worked on other tools (DFP, DCM, Media mind report, \n\nProficient in creation of queries in PL/SQL with proven abilities in the work, and Knowledge about VBA. \n\nAtlas reports, Flashtalking, Sizmek, Xad and Innovid). \n \n\nCareer Profile \n \nTata Consultancy Services \n \nDesignation \nEmployment Period \nDomain   \n \n➢  Project: Nielsen Statistical Analysis \n \nObjective: \n\n- Business Analyst \n- Nov 2013 to Jan 2017 \n- Retail Analytics \n\n \n\n \n\nThe Nielsen- world’s leading provider of marketing information, audience measurement, and business media products and services, \nbeing a part of Nielsen our aims is to provide clients with the most complete understanding of consumers and markets worldwide. \n \nRoles and responsibilities: \n\n•  Responsible for prompt  delivery of statistical counselling and support  for internal and external clients in accordance with \n\n• \n\n• \n• \n• \n• \n• \n• \n\nglobal statistical and technical standards. \nEnsure  proper  universe  updates  and  controls  are  implemented  at  intervals  consistent  with   client  global  and  regional \nstandards.  \nConduct regular coverage studies of client data to ensure our trends are accurate.  \nConduct trend analysis and forecasting provide written feedback within pre agreed timeframe.  \nCustomer loyalty analysis based on the Scan track data \nProviding Projection factors and inventory management reports based on seasonality co-efficient \nSampling of stores to perform studies and customer selection techniques for offering discounts. \nPrepared the country drossier template which includes all the process related information in one file in order to help the \nnew associates especially to get familiar with country specifics and will help them to catch up process quickly. \n\n•  Reporting  all  the  activities  and  the  challenges  in  the  team  on  a  daily  basis  and  weekly  bases  to  clients  and  supervisors  by \n\nconducting the weekly review call \n\n•  As a part of client enquiries team, took the additional responsibility of investigating the data to gain an understanding of the query \n\nand able to provide solution to the client. \n\n \n\n\x0c'][' ➢  Project : BASES (Booz Allen Sales Estimating System)  \n \nObjective: \n\nBASES is a sales estimation methodology for new product in branded consumer product innovation.  Nielsen‘s suite of Simulated Test \nMarketing (STM) tools determines whether our concepts go into development and get into the hands of consumers. \n\n \nRoles and responsibilities: \n\n• \n• \n\nPerforming Campaign Analysis for the Products and Discounts offered. \n\nPerform Sales volume forecasting based on Value Rating, Purchase Intent, Purchase Frequency and Purchase Volume algorithm \n\n•  Applied statistical sampling design concepts to make good sample design to meet the client expectations i.e. enough precision, \n\nminimum bias and accurate picture of retail sales across globe.  \n\n•  Worked  for  two  completely  different  regions  i.e.  Greater  China  and  Europe  and  gain  the  market  knowledge  as  well  as  the \n\nconsumer behaviour considerably. Experienced the interaction of diverse background.   \nCalculate sales effectiveness and Revenue reports for all retail departments \nTaken the responsibility of Ad-hoc task based on client requirements \n\n• \n• \n \nTheorem India Private Limited \n\n \n\n \n\nDesignation \nEmployment Period \nDomain   \n\n- Campaign Specialist \n- Jan 05, 2017 to Jan 05, 2018 \n\n                  - Digital Marketing \n\n \n\n➢  Project : GAMUT \n\n \n\nObjective: \n\nOnline Media reporting involves gathering/highlighting information about how well a campaign/account is performing so that the \nclient/account managers can take necessary actions to meet the goals whether it is ROI or traffic for a stipulated time/period. It is \nused for post-advertisement, historic data representation. \n \n\nRoles and Responsibilities:  \n\n• \n\nExperience in checking the billing status and fulfilling the campaigns using the first party and third party reports \n\n•  Working closely with the Associate Manager and assist in providing timely reporting and ensure production metrics are being \n\nmet on a daily basis \n\n• \n\nCreated an automation template to check the performance of the campaign and to maintain the accuracy in the report \n\n \n➢  Project : Process Centralization and Standardization \n\n \n\nObjective: \n\nCentralized training and standardization of work leave no scope for replication of tasks or actions. \nThis eliminates additional expenditure on excessive labour for duplication of work.  \nData standardization, on the other hand, is the process of transforming data (available in different formats) to a standard format, \nso as to enhance its efficiency and boost its filtering capability. \n\n \n\nRoles and Responsibilities:  \n\n•  Understanding & analysing the Project Requirement from client and do it in Systematic way \n•  Handling Advertiser end reporting. \n• \n\nPerforming Invoice Billing.  \n\n•  Quality Check for sub ordinates and coordinate and ensure the task is completed with 100% accuracy. \n• \n\nTo summarize, Centralization and standardized data allows us to: Easily group and filter data for segmenting your database, \nimprove the efficiency of customer acquisition and remain confident that your personalization is correct and that it can be \nexecuted without worrying about errors in the data. \n \n\n\x0c'][' Achievements: \n•  Received Team Spirit Award  \n•  Holds the distinction of being the Star Performer in 2015 and 2016 \n \nAcademic Qualifications \n \nMaster of Science (Statistics), 2013, Department of Statistics, Manasagangothri, University of Mysore, with 7.56 GPA  \n\n \nPersonal Details \n \nDate of Birth: \nAddress: \nLanguages Known: \nMarital Status                                   - Single \n\n \n\n \n\n \n\n- February 22nd, 1989 \n- #78 Maruthi Extension, Hunsur, Mysore District. \n- English, Kannada and Hindi \n\n\x0c'] "

In [56]:
text21

"Asha_Resume.pdf [' Asha H R \n\nMobile: +91 9538044096 \n\nE-Mail: ashahr.jc@gmail.com \n\nStatistical Analysis, Reporting, Market Research and Digital Marketing \n\n \nCareer Objective: \n \nTo secure a responsible position in Statistical Analytics/Modelling/SQL reporting where, I may utilize my professional skills and \npersonal capabilities towards enhancing the growth of the organization. \n \nProfile Summary: \n•  M.Sc. in Statistics with 3.2 years of experience in Statistical Analysis, Market Research and SQL reporting with Nielsen and \n\nBASES and 1 year of experience in online media reporting as a Campaign specialist with Gamut. \n\n•  Have proficient knowledge in analysing the data using some statistical tools like MS Excel, MS Word, MS Access PL/SQL and \n\nBase SAS and digital marketing tools like DFA, DFP, Flashtalking, Sizmek, Xad and TTD. \n \n\nTechnical Skillset: \n \n•  Working knowledge on Statistics concept, SAS, PL/SQL, and SPSS  \n•  Well versed with MS Office Su

In [57]:
extract_names(text21)

'Mysore District'

In [62]:
def extract_names1(document):
    nlp_text=nlp(document)
    #pattern=[{'POS':'PROPN'},{'POS':'PROPN'}]
    pattern1=[{'POS':'PROPN'},{'POS':'PROPN'},{'POS':'PROPN'}]
    matcher.add('NAME',None,pattern1)
    matches=matcher(nlp_text)
    for match_id,start,end in matches:
        span=nlp_text[start:end]             
        return span.text

In [63]:
extract_names1(text21)

'E-'